In [1]:
import requests
import json
import pandas as pd
import numpy as np


with open('city_list.json', encoding='utf-8') as f:
    d = json.load(f)

s_city = [i['name'] for i in d]
i = 0
for city in d:
    if city['name'] == '' or city['country'] == '':
        del s_city[i]
    i += 1
    
temperature = []
symbol_var = []
humidity = []

temperature_goal = []
symbol_var_goal = []

data_frame = pd.DataFrame()

appid = 'c0d78490bfcbfdc9dd77405dc48245d0'
for i in range(500):
    print(i)
    try:
        res = requests.get("http://api.openweathermap.org/data/2.5/forecast",
                     params={'q': s_city[i], 'units': 'metric', 'APPID': appid})

        data = res.json()
        object_8 = [data['list'][7 + 8*i] for i in range(5)]
#         for i in range(5):
#             data_frame['temp_' + str(i)] = object_8[i]['main']['temp']
#             data_frame['sumbol_var_' + str(i)] = object_8[i]['weather'][0]['icon']
        temperature_list = [obj['main']['temp'] for obj in object_8]
        humidity_list = [obj['main']['humidity'] for obj in object_8]
        symbol_var_list = [obj['weather'][0]['icon'] for obj in object_8]
        
        temperature.append(temperature_list)
        humidity.append(humidity_list)
        symbol_var.append(symbol_var_list)
        
        goal_object = data['list'][0]
        temperature_goal.append(goal_object['main']['temp'])
        symbol_var_goal.append(goal_object['weather'][0]['icon'])
    except Exception as e:
        print("Exception (find):", e)
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
Exception (find): 'list'
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
Exception (find): 'list'
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
Exception (find): 'list'
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
Exception (find): 'list'
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
25

In [2]:
for i in range(len(temperature_goal)):
    for i in range(5):
        data_frame['temp_' + str(i)] = [temp[i] for temp in temperature]
        data_frame['symbol_var_' + str(i)] = [symbol[i] for symbol in symbol_var]
        data_frame['hum_' + str(i)] = [hum[i] for hum in humidity]

In [3]:
data_frame.head()

,temp_0,symbol_var_0,hum_0,temp_1,symbol_var_1,hum_1,temp_2,symbol_var_2,hum_2,temp_3,symbol_var_3,hum_3,temp_4,symbol_var_4,hum_4
0,14.07,03d,78,16.06,03d,70,18.95,04d,55,19.95,04d,66,17.91,03d,86
1,24.47,03d,47,24.86,04d,39,24.55,01d,53,14.55,01d,71,12.71,01d,49
2,-0.74,13n,83,0.36,13n,93,0.65,13n,91,1.55,13n,90,1.61,13n,88
3,36.47,04n,18,38.36,01n,15,32.95,04n,29,38.15,03n,18,35.41,01n,19
4,15.87,01d,80,17.66,01d,70,19.75,03d,61,20.65,04d,65,17.61,01d,86


In [4]:
data_frame.isnull().values.any()

False

In [5]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split

data_frame['temp_goal'] = temperature_goal
data_frame['symbol_goal'] = symbol_var_goal


In [6]:
def label_encode(le, data, feature, feature_dict):
    try:
        feature_dict[feature]
        data[feature + '_le'] = data[feature].map(feature_dict[feature]).fillna(-1)
        data = data.drop([feature], axis=1)
    except KeyError:
        le.fit(data[feature].astype(str))
        data[feature + '_le'] = le.transform((data[feature].values))
        feature_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        data = data.drop([feature], axis=1)
        return feature_dict

def preprocessing_data(data):
    icon_dict = {'01d': 0, '04d': 1, '03d': 2, '10d': 3, '02d': 4, '10n': 5,
             '04n': 6, '01n': 7, '03n': 8, '13d': 9, '02n': 10}

    for i in range(5):
        data['symbol_var_' + str(i)] = data['symbol_var_' + str(i)].map(icon_dict)

    data['symbol_goal'] = data['symbol_goal'].map(icon_dict)   
    data = data.dropna()
    return data

data_frame = preprocessing_data(data_frame)

In [7]:

X = data_frame.drop(['temp_goal', 'symbol_goal'], axis=1)
y_temp = data_frame['temp_goal']
y_symb = data_frame['symbol_goal']

X_train, X_test, y_temp_train, y_temp_test, y_symb_train, y_symb_test = train_test_split(X, y_temp, y_symb, test_size=0.3, random_state=42)

In [8]:
regressor = LinearRegression()
regressor.fit(X_train, y_temp_train)
prediction = regressor.predict(X_test)

In [10]:
from sklearn.metrics import mean_absolute_error, median_absolute_error

print('The Explained Variance: %.2f' % regressor.score(X_test, y_temp_test))
print('The Mean Absolute Error: %.2f degrees celcius' % mean_absolute_error(
    y_temp_test, prediction))
print('The Median Absolute Error: %.2f degrees celcius' %
      median_absolute_error(y_temp_test, prediction))

The Explained Variance: 0.78
The Mean Absolute Error: 2.58 degrees celcius
The Median Absolute Error: 1.81 degrees celcius


In [11]:
from sklearn.ensemble import RandomForestClassifier


reg_tree = RandomForestClassifier(max_depth=9, max_features=3, n_estimators=200, random_state=42)
reg_tree.fit(X_train, y_symb_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [12]:
preds = reg_tree.predict(X_test)